# Jupyter Notebook - S3 Ransomware Response
This notebook is to be used to demonstrate how to respond to a ransomware attack on Amazon S3. 
Referencing https://github.com/aws-samples/aws-customer-playbook-framework/blob/main/docs/Ransom_Response_S3.md

The notebook will guide you in connecting and quering Amazon Athena databases and tables for the purpose of identifying suspicious S3 API calls, successful deletion of Amazon S3 Buckets and S3 Objects, and determining if exfiltration of data occured on a targeted Amazon S3 Bucket. 

**Authors: Tim Manik & Jeremy Wade**

*Note: This notebook assumes you have set up Amazon Security Lake within your AWS environment. You can replace the database and table names to match your own setup.*

# Setup

## Load Libraries

In order to query CloudTrail and interact with AWS, we need to load several libraries and configure our environment.

In [ ]:
pip install pyathena --quiet

In [ ]:
import boto3  # the Python SDK for AWS
import json
from datetime import datetime
import pandas as pd # Pandas is a data analysis tool for Python
from pyathena import connect # Python API client for Amazon Athena
region='us-east-1' # Set region variable to us-east-1 for API commands
athena_bucket = f's3://aws-athena-query-results-123456789123-{region}'  # S3 bucket that is configured to store your Athena queries
db_name = 'amazon_security_lake_glue_db_us_east_1' # database used by Athena. Choose 'default' if your CloudTrail was created via the Athena console and 'security_analysis' if you are using the AWS Security Analytics Bootstrap

Note: The following variables will need to be set to the aproriate values prior to running the cell: region, athena-bucket, db_name. 

## 1.0 Set up helper function for Athena

The Python query_results function shown below will help you query Athena tables.

In [ ]:
def query_results(sql):
    
    cursor = connect(s3_staging_dir=athena_bucket, region_name=region).cursor()
    cursor.execute(sql)
    
    columns = cursor.description
    data = cursor.fetchall()

    column_names = [column[0] for column in columns]
    rows = [dict(zip(column_names, data_row)) for data_row in data]

    df = pd.DataFrame(rows, columns=column_names)
    df1 = df.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
    
    return df1.set_properties(**{'text-align': 'center'})

# 1.1 Connect to CloudTrail

### Generic Query to Review Unfiltered Logs

It is recommend to conduct a simplified SQL query against unfiltered logs to ensure a connection is to the database and tables is made successfully. Additionally, an unfiltered request will provide an overview of the available data that can be queried. 

In [ ]:
generic = f"""
SELECT * 

FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_cloud_trail_mgmt_1_0" 
ORDER BY time
LIMIT 10
"""

In [ ]:
results = query_results(generic)
results

## 2.1 Review of Relevent GuardDuty Findings

The following queries are used to identify High Severity GuardDuty findings and a summary of most commong Tactics, Techniques, and Procedures (TTPs) related to the Amazon S3 service as a resource. 

### List high severity GuardDuty findings with S3 as the resource type

In [ ]:
highsev_s3 = f"""
SELECT
*
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_sh_findings_1_0" 
WHERE metadata.product.feature.name = 'GuardDuty'
AND severity = 'High'
AND resources[1].type = 'AwsS3Bucket'
ORDER BY time DESC
limit 10;
"""

In [ ]:
results = query_results(highsev_s3)
results

### Summarizes most common TTPs against S3

In [ ]:
highsev_s3_summary = f"""
SELECT
    count(*) as count,
    severity as severity,
    resources[1].type as resourceType,
    resources[1].uid as resourceId,
    finding.title as findingTitle
FROM
    "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_sh_findings_1_0"
WHERE
    metadata.product.feature.name = 'GuardDuty'
    AND severity = 'High'
    AND resources[1].type = 'AwsS3Bucket'
GROUP BY
    severity,
    resources[1].type,
    resources[1].uid,
    finding.title
ORDER BY
    count DESC
LIMIT 10;
"""

In [ ]:
results = query_results(highsev_s3_summary)
results

## 2.2 Review CloudTrail for suspicious S3 API calls

Next responders can start reviewing if any suspicious S3 API calls were made. Essentially, responders will be reviewing if there were any "impact" as part of the MITRE ATT&CK framework. Additional information regarding MITR ATT&CK: https://attack.mitre.org/matrices/enterprise/ 


In [ ]:
api = ['DeleteBucket','DeleteBucketCors', 'DeleteBucketEncryption','DeleteBucketLifecycle','DeleteBucketPolicy','DeleteBucketReplication','DeleteBucketTagging','DeleteBucketPublicAccessBlock']
t = tuple(api)

suspicious_apis = f"""
SELECT * 

FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_cloud_trail_mgmt_1_0" 
WHERE api.operation IN {t}
limit 5;
"""


In [ ]:
results = query_results(suspicious_apis)
results

### 2.2.1 Summarize logs of suspicious S3 API calls SUCCESSFULLY made


Generate a cleaner view of suspicious S3 API calls that were made successfully. 

In [ ]:
deleteApis = ['DeleteBucket','DeleteBucketCors', 'DeleteBucketEncryption','DeleteBucketLifecycle','DeleteBucketPolicy','DeleteBucketReplication','DeleteBucketTagging','DeleteBucketPublicAccessBlock']
t = tuple(deleteApis)

suspicious_apis_summary = f"""
SELECT accountid as AccountID, region as Region, time as Time, unmapped['requestParameters.bucketName'] as BucketName, api.operation as ApiCall, status as Status, actor.user.type as IdentityType, actor.session.issuer as RoleName, src_endpoint.ip as IpAddress, src_endpoint.uid as Vpc_Endpoint, src_endpoint.domain as AccessViaService 
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_cloud_trail_mgmt_1_0" 
WHERE api.operation IN {t}
AND status = 'Success'
ORDER BY time
limit 25;
"""

In [ ]:
results = query_results(suspicious_apis_summary)
results

### 2.2.2 Count of Suspicious S3 API Calls

Generate a count view of suspicious S3 API calls that were successfull.

In [ ]:
deleteApis = ['DeleteBucket','DeleteBucketCors', 'DeleteBucketEncryption','DeleteBucketLifecycle','DeleteBucketPolicy','DeleteBucketReplication','DeleteBucketTagging','DeleteBucketPublicAccessBlock']
t = tuple(deleteApis)

suspicious_apis_count = f"""
SELECT count(*) as Count, api.operation as ApiCall, api.service.name as Service

FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_cloud_trail_mgmt_1_0" 
WHERE api.operation IN {t}
AND status = 'Success'
GROUP BY api.operation, api.service.name
ORDER BY Count Desc
limit 25;
"""

In [ ]:
results = query_results(suspicious_apis_count)
results

### 2.2.3 Deleted Objects and Buckets

This query shows an overview of successfull attempts at deleting an Amazon S3 Bucket and S3 Objects. 

In [ ]:
deleteObjBuc = ['DeleteBucket','DeleteObject']
t = tuple(deleteObjBuc)

deletedFiles = f"""
SELECT accountid as AccountID, region as Region, unmapped['requestParameters.bucketName'] as BucketName, api.operation as ApiCall, status as Status, actor.user.type as IdentityType, actor.session.issuer as RoleName, src_endpoint.ip as IpAddress, src_endpoint.uid as Vpc_Endpoint, src_endpoint.domain as AccessViaService 
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_cloud_trail_mgmt_1_0" 
WHERE api.operation IN {t}
AND status = 'Success'
GROUP BY accountid, region, unmapped['requestParameters.bucketName'], api.operation, status, actor.user.type, actor.session.issuer, src_endpoint.ip, src_endpoint.uid, src_endpoint.domain
limit 25;
"""

In [ ]:
results = query_results(deletedFiles)
results

#### 2.2.3.1 Lists Buckets that are SUCCESSFULLY deleted

In [ ]:
delete_bucket_count = f"""
SELECT
api.operation as ApiCall, api.service.name as Service, unmapped['requestParameters.bucketName'] as BucketName, resources, actor.user.type as IdentityType, actor.session.issuer as RoleName
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_cloud_trail_mgmt_1_0" 
WHERE api.operation = 'DeleteBucket'
AND status = 'Success'
GROUP BY api.operation, api.service.name, unmapped['requestParameters.bucketName'], resources, actor.user.type, actor.session.issuer
limit 25;
"""

In [ ]:
results = query_results(delete_bucket_count)
results

#### 2.2.3.2 List Objects that are SUCCESSFULLY deleted

In [ ]:
delete_obj_count = f"""
SELECT
api.operation as ApiCall, api.service.name as Service, unmapped['requestParameters.bucketName'] as BucketName, resources, actor.user.type as IdentityType, actor.session.issuer as RoleName
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_s3_data_1_0" 
WHERE api.operation = 'DeleteObject'
AND status = 'Success'
GROUP BY api.operation, api.service.name, unmapped['requestParameters.bucketName'], resources, actor.user.type, actor.session.issuer
limit 10;
"""

In [ ]:
results = query_results(delete_obj_count)
results

## 2.3 Investigate Cloudtrail for Suspicious Identities

The following queries can be used to investigate suspicious identities to establish other activities conducted leveraging other API operations. 

*Note 1: Replace the value of {role_arn} with the ARN of the identity you would like to investigate.*  
*Note 2: The query can be copied and repeated for each identity that needs to be investigated. You can increment the {role_arn} variable, or create a new variable.*


In [ ]:
role_arn1 = 'IAM USER ARN'

In [ ]:
suspicious_role1 = f"""
SELECT
time, http_request, src_endpoint, actor.user.type as IdentityType, actor.session.issuer as RoleName, api.operation as ApiCall, api.service.name as Service, status, resources
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_cloud_trail_mgmt_1_0" 
WHERE actor.session.issuer = '{role_arn1}'
GROUP BY time, http_request, src_endpoint, actor.user.type, actor.session.issuer, api.operation, api.service.name, status, resources
ORDER BY time ASC
limit 25;
"""

In [ ]:
results = query_results(suspicious_role1)
results

## 2.2 EXFILTRATION

### Amount of data transferred to a specific IP address

Show a list of IP addresses that data has been transferred out to from S3

In [ ]:

data_transfers = f"""

SELECT coalesce(SUM(CAST(unmapped['additionalEventData.bytesTransferredOut'] AS DECIMAL(30,2))), 0)/1000000000 AS GigabytesSentTotal, accountid as AccountID, region as Region, unmapped['requestParameters.bucketName'] as BucketName, src_endpoint.ip as IpAddress
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_s3_data_1_0"
WHERE CAST(unmapped['additionalEventData.bytesTransferredOut'] AS DECIMAL(30,2)) > 0
AND src_endpoint.ip IS NOT NULL
GROUP BY src_endpoint.ip, accountid, region, unmapped['requestParameters.bucketName'], src_endpoint.uid, src_endpoint.domain
ORDER BY GigabytesSentTotal DESC
Limit 10;

"""

In [ ]:
results = query_results(data_transfers)
results

### Top 5 Largest Data Transfer Out

In [ ]:
data_transfers_summary = f"""

SELECT coalesce(SUM(CAST(unmapped['additionalEventData.bytesTransferredOut'] AS DECIMAL(30,2))), 0)/1000000000 AS GigabytesSentTotal, src_endpoint.ip as IpAddress
FROM "amazon_security_lake_glue_db_us_east_1"."amazon_security_lake_table_us_east_1_s3_data_1_0"
WHERE CAST(unmapped['additionalEventData.bytesTransferredOut'] AS DECIMAL(30,2)) > 0
AND src_endpoint.ip IS NOT NULL
GROUP BY src_endpoint.ip
ORDER BY GigabytesSentTotal DESC
Limit 5;

"""

In [ ]:
results = query_results(data_transfers_summary)
results